In [1]:
import numpy as np
import pandas as pd
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_dataset
from numpy import genfromtxt
from logging import INFO
import xgboost as xgb
from flwr.common import log
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner
from pandas import concat

/home/zuy/anaconda3/envs/flower_tutorial/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-22 10:20:34,714	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
df = pd.read_csv('/home/zuy/Documents/BCU/ML/CSV/df_train_3.csv')
    df.drop("Unnamed: 0", axis=1, inplace=True)

In [5]:
df.shape

(227845, 18)

In [38]:
df.Class.value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [7]:
"""Load partition HIGGS data."""
# Only initialize `FederatedDataset` once
fds = None  # Cache FederatedDataset
if fds is None:
    partitioner = IidPartitioner(num_partitions=1)
    fds = FederatedDataset(
        dataset="jxie/higgs",
        partitioners={"train": partitioner},
    )
    partition = fds.load_partition(0, split="train")
    partition.set_format("numpy")

Generating val_16k split: 100%|██████████| 15688/15688 [00:00<00:00, 407005.93 examples/s]


In [ ]:
"""Load partition HIGGS data."""
# Only initialize `FederatedDataset` once
fds = None  # Cache FederatedDataset
if fds is None:
    partitioner = IidPartitioner(num_partitions=1)
    fds = FederatedDataset(
        dataset="ylecun/mnist",
        partitioners={"train": partitioner},
    )
    
# Load the partition for this `partition_id`
partition = fds.load_partition(0, split="train").set_format("numpy")
# Train/test splitting
train_data, valid_data, num_train, num_val = train_test_num_split(partition, test_fraction=0.2, seed=42)

In [ ]:
"""Load partition df_3 data."""
df = pd.read_csv('/home/zuy/Documents/BCU/ML/CSV/df_3.csv')
df.drop("Unnamed: 0", axis=1, inplace=True)
dataset = Dataset.from_pandas(df)
partitioner = IidPartitioner(num_partitions=1)
partitioner.dataset = dataset
dataset = partitioner.load_partition(partition_id=0).to_pandas()

X = dataset.drop('Class', axis=1)
y = dataset['Class']
# new_data = xgb.DMatrix(X, label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, random_state=42, stratify=y)
partition_train = pd.concat([X_train, X_test])
partition_test = pd.concat([y_train, y_test])
num_train = len(partition_train)
num_test = len(partition_test)


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [3]:
df = pd.read_csv('/home/zuy/Documents/BCU/ML/CSV/df_train_3.csv')
df.drop("Unnamed: 0", axis=1, inplace=True)

In [4]:
df.head(2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,Amount,Class
0,0.905774,0.862468,-0.582668,-0.800214,-0.402839,1.195050,3.686711,-1.352372,2.084633,1.176746,-0.064710,-0.054899,0.360069,0.057693,-0.126514,-0.204315,0.0
1,0.465984,0.902013,-0.081009,-1.688308,0.169440,2.300397,3.063923,-0.037911,1.216367,0.077896,0.381968,-0.149059,0.179281,-0.298349,0.731565,-0.264579,0.0


In [5]:
df_new = MinMaxScaler(feature_range=(0, 1)).fit_transform(df)
df_new = pd.DataFrame(df_new, columns=[
'Time','V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','Amount','Class'])

In [6]:
df_new.head(2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,Amount,Class
0,0.937075,0.991418,0.759318,0.813931,0.222601,0.775833,0.303487,0.256341,0.797613,0.511242,0.505465,0.278523,0.723425,0.453073,0.643815,0.000285,0.0
1,0.720386,0.992920,0.766743,0.784411,0.262921,0.785532,0.296185,0.265342,0.792624,0.464359,0.514616,0.270128,0.716463,0.416956,0.670298,0.000116,0.0


In [7]:
df_new['Class'].unique()

array([0., 1.])

In [8]:
temp_class = df_new['Class']
temp_df = df_new.drop('Class', axis=1)
temp_df = temp_df * 20000
temp_df.head(2)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,Amount
0,18741.492662,19828.356443,15186.362165,16278.623253,4452.014003,15516.664968,6069.734946,5126.825088,15952.256295,10224.846798,10109.307540,5570.452646,14468.502119,9061.453421,12876.306388,5.698458
1,14407.727210,19858.394983,15334.852160,15688.216530,5258.427845,15710.635174,5923.702380,5306.843169,15852.475389,9287.170578,10292.314406,5402.552065,14329.261237,8339.117567,13405.959811,2.327649


In [9]:
temp_class = pd.DataFrame(temp_class, columns=["Class"])
temp_df2 = pd.concat([temp_df, temp_class], axis=1)
temp_df2.head(2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,Amount,Class
0,18741.492662,19828.356443,15186.362165,16278.623253,4452.014003,15516.664968,6069.734946,5126.825088,15952.256295,10224.846798,10109.307540,5570.452646,14468.502119,9061.453421,12876.306388,5.698458,0.0
1,14407.727210,19858.394983,15334.852160,15688.216530,5258.427845,15710.635174,5923.702380,5306.843169,15852.475389,9287.170578,10292.314406,5402.552065,14329.261237,8339.117567,13405.959811,2.327649,0.0


In [10]:
temp_df2.max()

Time      20000.0
V1        20000.0
V2        20000.0
V3        20000.0
V4        20000.0
V5        20000.0
V6        20000.0
V7        20000.0
V8        20000.0
V9        20000.0
V10       20000.0
V11       20000.0
V12       20000.0
V13       20000.0
V14       20000.0
Amount    20000.0
Class         1.0
dtype: float64

In [11]:
temp_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227845 entries, 0 to 227844
Data columns (total 17 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    227845 non-null  float64
 1   V1      227845 non-null  float64
 2   V2      227845 non-null  float64
 3   V3      227845 non-null  float64
 4   V4      227845 non-null  float64
 5   V5      227845 non-null  float64
 6   V6      227845 non-null  float64
 7   V7      227845 non-null  float64
 8   V8      227845 non-null  float64
 9   V9      227845 non-null  float64
 10  V10     227845 non-null  float64
 11  V11     227845 non-null  float64
 12  V12     227845 non-null  float64
 13  V13     227845 non-null  float64
 14  V14     227845 non-null  float64
 15  Amount  227845 non-null  float64
 16  Class   227845 non-null  float64
dtypes: float64(17)
memory usage: 29.6 MB


In [14]:
embedding = nn.Embedding(num_embeddings=20001, embedding_dim=8)
# df -> torch tensor (LongTensor)
input_tensor = torch.LongTensor(temp_df2.values)
embedded_output = embedding(input_tensor)  # Shape: [36455, 17, 8]
print(embedded_output.shape)

torch.Size([227845, 17, 8])


In [ ]:
class MultiheadAttention(nn.Module):
    def __init__(self, input_dim, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        # now three separate linears instead of one combined qkv_layer
        self.q_linear = nn.Linear(input_dim, d_model)
        self.k_linear = nn.Linear(input_dim, d_model)
        self.v_linear = nn.Linear(input_dim, d_model)

        self.output_linear = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        # x: [B, S, input_dim]
        B, S, _ = x.size()

        # project to queries, keys, values
        Q = self.q_linear(x)  # [B, S, d_model]
        K = self.k_linear(x)
        V = self.v_linear(x)

        # reshape + split into heads
        # → [B, S, num_heads, head_dim]
        Q = Q.view(B, S, self.num_heads, self.head_dim).transpose(1, 2)  # [B, H, S, Dh]
        K = K.view(B, S, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(B, S, self.num_heads, self.head_dim).transpose(1, 2)

        # scaled dot-product attention per head
        values, attn = scaled_dot_product(Q, K, V, mask)  # both [B, H, S, Dh]

        # recombine heads
        values = values.transpose(1, 2).contiguous().view(B, S, self.d_model)  # [B, S, d_model]

        # final output projection
        out = self.output_linear(values)  # [B, S, d_model]
        return out, attn